In [1]:
using ITensors
using JLD2

## Task 3.1

In [6]:
ITensors.set_warn_order(25);

In [7]:
function entropies(ψ::MPS)
    L = collect(2:length(ψ))
    S_arr = Vector{Float64}()
    S_bi = Vector{Float64}()
    
    for b in L
        orthogonalize!(ψ, b)
        U,S,V = svd(ψ[b], b > 1 ? (linkind(ψ, b-1), siteind(ψ,b)) : (siteind(ψ,b))) 
        SvN = 0.0
        #normalize!(S)
        for k = 1:dim(S, 1)
            p = S[k,k]^2
            SvN -= p*log(p)
            if b == Int(floor(maximum(L)/2))
                push!(S_bi, S[k,k])
            end
        end
        push!(S_arr, SvN)
    end

    return S_arr, S_bi
end

entropies (generic function with 1 method)

In [14]:
S_arr = Vector{Vector{Float64}}()
S_bi = Vector{Vector{Float64}}()
for L in [10, 15, 20]
    sites = siteinds("S=1/2",L)
    T = randomITensor(sites)
    M = MPS(T, sites)
    ψ = M/norm(M)
    S, S_c = entropies(ψ::MPS)
    pushfirst!(S, 0.0)
    push!(S_arr, S)
    push!(S_bi, S_c)
end
save_object("EntanglementEntropies.jld2", S_arr)
save_object("EntanglementSpectrum.jld2", S_bi)

In [12]:
S_bi

3-element Vector{Vector{Float64}}:
 [0.7553821716761205, 0.5200437860298722, 0.35710534669738103, 0.1772794592887968]
 [0.33384959931562813, 0.3086501683499409, 0.28696948577809567, 0.27548550724863125, 0.2659431789186319, 0.2555834374540786, 0.2534690299646473, 0.2298325389279834, 0.214262503899654, 0.21276650304579248  …  0.0819420947426108, 0.07705711212819244, 0.05688123095901979, 0.05120257825613731, 0.043512882442064575, 0.03715987853511496, 0.02504534221105691, 0.019519136870872686, 0.007097008427543401, 0.0016905443646798954]
 [0.06229042621987295, 0.061899398759054036, 0.06163883161808247, 0.06124187031583402, 0.06085378645204433, 0.06077371725704941, 0.06064866924934978, 0.060632142012519895, 0.06037410059528212, 0.0602087691384744  …  0.00036702019589969655, 0.0003433500020534791, 0.0003246243554314986, 0.0002618535843372374, 0.0002289550662962062, 0.00020881170536233033, 0.0001431091827270288, 0.000114511770601754, 6.368049853091261e-5, 1.2283383692179007e-5]